# Intro to Quantum Computing with Python and Q\# <br> [//build 2019](https://mybuild.techcommunity.microsoft.com/sessions/77163) #

Let's use Python interoperability for the [Quantum Development Kit](https://github.com/Microsoft/Quantum) to introduce users to Q\# and quantum computing more generally.

---


Installation instructions for running this notebook on your machine can be found [here](https://docs.microsoft.com/en-ca/quantum/install-guide/python?view=qsharp-preview).

Let's get started by loading the packages we need, including the Q# interoperability package `qsharp`.

In [37]:
import qsharp
#qsharp.reload()
qsharp.component_versions()

{'iqsharp': LooseVersion ('0.6.1905.301'),
 'Jupyter Core': LooseVersion ('1.1.13141.0'),
 'qsharp': LooseVersion ('0.6.1905.301')}

In [20]:
qsharp.reload()

## What is a qubit?

A single qubit can be represented by a vector of 2 complex numbers* like this:

$\left|{x}\right\rangle = \left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

<small>* some conditions apply</small>

## What can we _do_ with a qubit?

You can do three types of things with qubits:

- Prepare a qubit
- Do operations with a qubit
- Measure a qubit

### Preparing a qubit

In [38]:
prepare_qubit = qsharp.compile("""
open Microsoft.Quantum.Diagnostics;

operation PrepareQubit() : Unit {
    using (qubit = Qubit()) {     // We want 1 qubit to use for our task
            DumpMachine();         // Print out what the simulator is keeping a record of
    }
}
""")

In [39]:
prepare_qubit.simulate()

# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0


()

You can read the above output like the vector we wrote above, where the first column is the index, the second is the real part of the vector at that position, and the second is the complex part of that vector entry.

What does `DumpMachine` tell us?

```
# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0
```

This is the same ket we saw earlier!

$\left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

## Case Study: Generating quantum random numbers

```Q#
// build-demo.qs
namespace Build.Demo {
    operation Qrng() : Result {
        using (qubit = Qubit()) {
            H(qubit);
            return MResetZ(qubit);
        }
    }
}
```

We can see that this is a '.qs' file, so it is a Q# source file and it has a number of operations and functions we can import:

In [47]:
from Build.Demo import Qrng, QrngVerbose

## Understanding `Qrng`

We can use built-in documentation as a resource.

In [59]:
?Qrng

That tells us what we can **do** with `Qrng`:

In [60]:
[Qrng.simulate() for _ in range(10)]

[1, 1, 0, 1, 0, 0, 1, 0, 1, 0]

### Randomness brought to you by superposition!

We can use `DumpMachine` again to understand see what the `H` operation does to our qubit.

In [61]:
QrngVerbose.simulate()

Here is what the simulator uses to record a qubit in the 0 state:
# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0
 
After using H(qubit) to create a superposition state:
# wave function for qubits with ids (least to most significant): 0
0:	0.707106781186548	0
1:	0.707106781186548	0


0

## Operation create entanglement

Using Q# with Python, we can also explore other quantum development concepts, like **entanglement**.

In [62]:
from Build.Demo import EntangleQubits
results = EntangleQubits.simulate(verbose=True)

State of inital two qubits:
# wave function for qubits with ids (least to most significant): 0;1
0:	1	0
1:	0	0
2:	0	0
3:	0	0
 
After entangling the two qubits:
# wave function for qubits with ids (least to most significant): 0;1
0:	0.707106781186548	0
1:	0	0
2:	0	0
3:	0.707106781186548	0


What does `DumpRegister` tell us this time?
```
# wave function for qubits with ids (least to most significant): 0;1
0:	0.707106781186548	0
1:	0	0
2:	0	0
3:	0.707106781186548	0
```
No matter how many times we run, both measurements are equal to each other.

In [63]:
[EntangleQubits.simulate(verbose=False) for _ in range(10)]

[(1, 1),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1)]

---

## Toy quantum algorithim: Deutsch–Jozsa 

_If I had a function that had one bit input and ouput, how many different options would I have?_

<figure style="text-align: center;">
    <img src="media/twobit.png" width="60%">
    <caption>
      <br>  
        <strong>Diagram of all possible one bit functions</strong>
    </caption>
</figure>

>#### Deutsch–Jozsa Algorithim ####
>**Problem statement:**
>
>* **GIVEN:** A black box quantum operation which takes 1 input bit and produces either a 0 or a 1 as output. We are promised that the box is either _constant_ or _balanced_. 
>					
>* **GOAL:** to determine if the box output is _constant_ or _balanced_ by evaluating sample inputs.

<figure style="text-align: center;">
    <img src="media/twobitDJ.png" width="50%">
    <caption>
      <br>  
        <strong>Global property of the one bit functions: Constant or Balanced</strong>
    </caption>
</figure>

In [11]:
available_ops['Build.DeutschJozsa']

['IdOracle',
 'IsNotOracleBalanced',
 'IsOracleBalanced',
 'IsZeroOracleBalanced',
 'NotOracle',
 'OneOracle',
 'RunDeutschJozsaAlgorithm',
 'ZeroOracle']

In [17]:
is_zero_oracle_balanced = qsharp.compile("""
open Build.DeutschJozsa;

operation IsZeroOracleBalanced(): Bool {
    return IsOracleBalanced(ZeroOracle);
}
""")

In [18]:
is_zero_oracle_balanced.simulate()

False

In [19]:
is_not_oracle_balanced = qsharp.compile("""
open Build.DeutschJozsa;

operation IsNotOracleBalanced(): Bool {
    return IsOracleBalanced(NotOracle);
}
""")

In [20]:
is_not_oracle_balanced.simulate()

True

In [22]:
from Build.DeutschJozsa import RunDeutschJozsaAlgorithm, RunDeutschJozsaAlgorithmVerbose

In [46]:
RunDeutschJozsaAlgorithm.simulate()

All tests passed!


()

In [23]:
RunDeutschJozsaAlgorithmVerbose.simulate()

The ZeroOracle is Balanced: False
The OneOracle is Balanced: False
The IdOracle is Balanced: True
The NotOracle is Balanced: True
All tests passed!


()

---

## Helpful diagnostics :)

In [5]:
for component, version in sorted(qsharp.component_versions().items(), key=lambda x: x[0]):
    print(f"{component:20}{version}")

Jupyter Core        1.1.13141.0
iqsharp             0.6.1905.301
qsharp              0.6.1905.301


In [6]:
import sys
print(sys.version)

3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]
